## 株価データベースの作成のためのチュートリアル

In [1]:
import pandas as pd
from pathlib import Path
import sqlite3
import numpy as np

In [2]:
import datetime

### datatimeデータのsqliteへの利用 

connect関数の引数におまじないを与える．これでsqliteの型としてtimestampとするだけで，datetimeを文字列に変換，逆変換してくれる．

In [3]:
db_path = Path("pandas_db") / Path("pandas.db")
conn = sqlite3.connect(db_path,
                       detect_types=sqlite3.PARSE_DECLTYPES|sqlite3.PARSE_COLNAMES
                      )

timestamp型を利用する．一応ユニーク制約をつけておく

In [8]:
c = conn.cursor()
c.execute("create table time_table(timestamp timestamp unique, temperature real)")

In [9]:
time_data_list = [["2020-10-22 06:28:00",12.5],
                  [datetime.datetime(2020, 10, 22, 6, 29, 00), 13],
                  [datetime.datetime(2020, 10, 23, 0, 00, 00), 14]
                 ]

c.executemany("insert into time_table (timestamp,temperature) values(?,?)",
              time_data_list)

conn.commit()

日時で降順ソートしてみる

In [10]:
c.execute("select * from time_table order by timestamp desc")
result = c.fetchall()
print(result)

[(datetime.datetime(2020, 10, 23, 0, 0), 14.0), (datetime.datetime(2020, 10, 22, 6, 29), 13.0), (datetime.datetime(2020, 10, 22, 6, 28), 12.5)]


In [11]:
print(type(result[0][0]))

<class 'datetime.datetime'>


datetimeになっている

ちゃんと同じ時間にinsertするとエラーが出てくる．

In [12]:
c.execute("insert into time_table (timestamp,temperature) values(?,?)", 
          [datetime.datetime(2020, 10, 22, 6, 29, 0), 14])

IntegrityError: UNIQUE constraint failed: time_table.timestamp

期間指定による取り出しができる

In [12]:
c.execute("select * from time_table where ? <= timestamp and timestamp <= ?",
          ["2020-10-22 00:00:00", "2020-10-22 23:59:59"])
result = c.fetchall()
print(result)

[(datetime.datetime(2020, 10, 22, 6, 28), 12.5), (datetime.datetime(2020, 10, 22, 6, 29), 13.0)]


betweenをつかったり，指定にdatetimeを使ってもよい

In [13]:
c.execute("select * from time_table where timestamp between ? and ?",
          [datetime.datetime(2020,10,22,0,0,0), datetime.datetime(2020,10,22,23,59,59)]
         )
result = c.fetchall()
print(result)

[(datetime.datetime(2020, 10, 22, 6, 28), 12.5), (datetime.datetime(2020, 10, 22, 6, 29), 13.0)]


c.executeのパラメータを利用せず，文字列として与えることもできる．

In [21]:
sql = "select * from time_table where '{}' <= timestamp and timestamp < '{}'".format(datetime.datetime(2020,10,22,0,0,0), 
                                                                                                 datetime.datetime(2020,10,22,23,59,59))
c.execute(sql)
result = c.fetchall()
print(result)

[(datetime.datetime(2020, 10, 22, 6, 28), 12.5), (datetime.datetime(2020, 10, 22, 6, 29), 13.0)]


In [14]:
c.execute("select max(timestamp) from time_table")
result = c.fetchone()
str_datetime = result[0]
print(str_datetime)

2020-10-23 00:00:00


なぜかmaxやminなどの関数を通すと文字列で返ってくる．

### pandasによるto_sqlite

#### 通常のインデックスの場合

In [11]:
c.execute("create table time_table_pandas(timestamp timestamp unique, temperature real)") 

In [12]:
time_temperature = [[datetime.datetime(2020, 10, 22, 12, 30, 00), 13.5],
                    [datetime.datetime(2020, 10, 22, 12, 31, 00), 14.0],
                    [datetime.datetime(2020, 10, 22, 12, 32, 00), 12.0]]

time_temperature = np.array(time_temperature)
df = pd.DataFrame(time_temperature, columns=["timestamp", "temperature"])

In [13]:
df.to_sql("time_table_pandas", 
          con=conn, 
          if_exists="replace",
          dtype={"timestamp":"timestamp", "temperature":"real"},
         )

うまくいっているが，もちろんインデックスも返ってくる. インデックスを指定したくないときは`index=False`とする

In [14]:
c.execute("select * from time_table_pandas")
result = c.fetchall()
print(result)

[(0, datetime.datetime(2020, 10, 22, 12, 30), 13.5), (1, datetime.datetime(2020, 10, 22, 12, 31), 14.0), (2, datetime.datetime(2020, 10, 22, 12, 32), 12.0)]


#### 時系列インデックスの場合

In [16]:
c.execute("create table time_index_table_pandas(timestamp timestamp unique, temperature real)") 

In [17]:
timestamp_df = pd.DataFrame(time_temperature[:,1],columns=["temperature"])
timestamp_df.index = pd.DatetimeIndex(time_temperature[:,0])

In [18]:
timestamp_df.to_sql("time_index_table_pandas", 
                    con=conn, 
                    if_exists="replace",
                    dtype={"temperature":"real"},
                    index=True,
                    index_label="timestamp"
                    )

インデックスの型指定は無い，これはpandas側の型指定と対応しているから?index_labelにはindexのcolumn名を指定する．

In [19]:
c.execute("select * from time_index_table_pandas")
result = c.fetchall()
print(result)

[(datetime.datetime(2020, 10, 22, 12, 30), 13.5), (datetime.datetime(2020, 10, 22, 12, 31), 14.0), (datetime.datetime(2020, 10, 22, 12, 32), 12.0)]


つまり，時系列インデックスの場合は普通のインデックスに直して`index=False`とするか，`to_sql`でインデックスに名前を指定すればよい

### pandasのsqliteの直接読み込み

クエリのfetchallで返ってきたリストをpandasに与えればよいのであまり必要性を感じないが直接読み込む関数が存在する．`read_sql_query`と`read_sql`の違いは？

In [26]:
query_df = pd.read_sql_query("select * from time_index_table_pandas where temperature = ?",
                             con=conn,
                             index_col="timestamp",
                             params=[13.5,]
                            )

In [28]:
query_df

,temperature
timestamp,
2020-10-22 12:30:00,13.5


### テーブルの削除 

In [22]:
c.execute("drop table time_table")

In [6]:
c.execute("drop table time_table_pandas")

OperationalError: no such table: time_table_pandas

In [7]:
c.execute("drop table time_index_table_pandas")

OperationalError: no such table: time_index_table_pandas

###  データベースのクローズ

In [23]:
conn.close()